In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/fix-colab-gpu.sh && bash fix-colab-gpu.sh

## Prepare Java Kernel for Google Colab
Since Java is not natively supported by Colab, we need to run the following code to enable Java kernel on Colab.

1. Run the cell bellow (click it and press Shift+Enter),
2. (If training on CPU, skip this step) If you want to use the GPU with MXNet in DJL 0.10.0, we need CUDA 10.1 or CUDA 10.2.
Since Colab supports CUDA 10.1, we will have to follow some steps to setup the environment.
Refresh the page (press F5) and stay at Python runtime on GPU. Run the file fix-colab-gpu script.

And then ensure that you have switched to CUDA 10.1.
3. After that, switch runtime to Java and hardware to GPU.(Might require refreshing the page and switching runtime)

Now you can write Java code.

In [ ]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh

# Homework 3     

Use Tablesaw to plot graphs.

In [ ]:
%load ../../utils/djl-imports
%load ../../utils/plot-utils
%load ../../utils/Functions

In [ ]:
import org.apache.commons.lang3.ArrayUtils;

NDManager manager = NDManager.newBaseManager();

# 1. Logistic Regression for Binary Classification

In multiclass classification we typically use the exponential model 

$$p(y|\mathbf{o}) = \mathrm{softmax}(\mathbf{o})_y = \frac{\exp(o_y)}{\sum_{y'} \exp(o_{y'})}$$

1.1. Show that this parametrization has a spurious degree of freedom. That is, show that both $\mathbf{o}$ and $\mathbf{o} + c$ with $c \in \mathbb{R}$ lead to the same probability estimate.
1.2. For binary classification, i.e. whenever we have only two classes $\{-1, 1\}$, we can arbitrarily set $o_{-1} = 0$. Using the shorthand $o = o_1$ show that this is equivalent to 

$$p(y=1|o) = \frac{1}{1 + \exp(-o)}$$

1.3. Show that the log-likelihood loss (often called logistic loss) for labels $y \in \{-1, 1\}$ is thus given by 

$$-\log p(y|o) = \log (1 + \exp(-y \cdot o))$$

1.4. Show that for $y = 1$ the logistic loss asymptotes to $o$ for $o \to \infty$ and to $\exp(o)$ for $o \to -\infty$. 

# 2. Logistic Regression and Autograd

1. Implement the binary logistic loss $l(y,o) = \log (1 + \exp(-y \cdot o))$ in DJL
1. Plot its values for $y \in \{-1, 1\}$ over the range of $o \in [-5, 5]$.
1. Plot its derivative with respect to $o$ for $o \in [-5, 5]$ using 'GradientCollector'.

In [ ]:
float loss(float y, float o) {
    // Add your loss function here
    float l = 0;
    return l;
}

# 3. Ohm's Law

Imagine that you're a young physicist, maybe named [Georg Simon Ohm](https://en.wikipedia.org/wiki/Georg_Ohm), trying to figure out how current and voltage depend on each other for resistors. You have some idea but you aren't quite sure yet whether the dependence is linear or quadratic. So you take some measurements, conveniently given to you as 'NDArrays' in DJL. They are indicated by 'current' and 'voltage'.

Your goal is to use least mean squares regression to identify the coefficients for the following three models using automatic differentiation and least mean squares regression. The three models are:

1. Quadratic model where $\mathrm{voltage} = c + r \cdot \mathrm{current} + q \cdot \mathrm{current}^2$.
1. Linear model where $\mathrm{voltage} = c + r \cdot \mathrm{current}$.
1. Ohm's law where $\mathrm{voltage} = r \cdot \mathrm{current}$.

In [ ]:
var current = manager.create(new float[]{1.5420291f, 1.8935232f, 2.1603365f, 2.5381863f, 2.893443f,
                    3.838855f, 3.92542f, 4.223369f, 4.235571f, 4.273397f,
                    4.9332876f, 6.4704757f, 6.517571f, 6.87826f, 7.0009003f, 
                    7.035741f, 7.278681f, 7.7561755f, 9.121138f, 9.728281f});
var voltage = manager.create(new float[]{63.802246f, 80.036026f, 91.4903f, 108.28776f, 122.781975f,
                    161.36314f, 166.50816f, 176.16772f, 180.29395f, 179.09758f,
                    206.21027f, 272.71857f, 272.24033f, 289.54745f, 293.8488f,
                    295.2281f, 306.62274f, 327.93243f, 383.16296f, 408.65967f})

# 4. Entropy

Let's compute the *binary* entropy of a number of interesting data sources. 

1. Assume that you're watching the output generated by a [monkey at a typewriter](https://en.wikipedia.org/wiki/File:Chimpanzee_seated_at_typewriter.jpg). The monkey presses any of the $44$ keys of the typewriter at random (you can assume that it has not discovered any special keys or the shift key yet). How many bits of randomness per character do you observe?
1. Unhappy with the monkey you replaced it by a drunk typesetter. It is able to generate words, albeit not coherently. Instead, it picks a random word out of a vocabulary of $2,000$ words. Moreover, assume that the average length of a word is $4.5$ letters in English. How many bits of randomness do you observe now?
1. Still unhappy with the result you replace the typesetter by a high quality language model. These can obtain perplexity numbers as low as 20 points per character. The perplexity is defined as a length normalized probability, i.e.

$$\mathrm{PPL}(x) = \left[p(x)\right]^{1/\mathrm{length}(x)}$$

# 5. Wien's Approximation for the Temperature (bonus)

We will now abuse DJL to estimate the temperature of a black body. The energy emanated from a black body is given by Wien's approximation.

$$B_\lambda(T) = \frac{2 h c^2}{\lambda^5} \exp\left(-\frac{h c}{\lambda k T}\right)$$

That is, the amount of energy depends on the fifth power of the wavelength $\lambda$ and the temperature $T$ of the body. The latter ensures a cutoff beyond a temperature-characteristic peak. Let us define this and plot it.

In [ ]:
// Lightspeed
float c = 299792458;
// Planck's constant
float h = (float) 6.62607004e-34;
// Boltzmann constant
float k = (float) 1.38064852e-23;
// Wavelength scale (nanometers)
float lamscale = (float) 1e-6;
// Pulling out all powers of 10 upfront
float pOut = 2 * h * (float) Math.pow(c, 2) / (float) Math.pow(lamscale, 5);
float pIn = (h / k) * (c / lamscale);

// Wien's law
NDArray wien(NDArray lam, float t) {
    return lam.pow(-5).mul(pOut).mul(lam.mul(t).pow(-1).mul(-pIn).exp());
}

// Plot the radiance for a few different temperatures
var lam = manager.arange(0, 100, 0.01f);
float[] lamArray = lam.toFloatArray();

// To hold all data
float[] lambdas = new float[0];
float[] radiances = new float[0];
String[] T = new String[0];

for (float t : new float[]{10, 100, 150, 200, 250, 300, 350}) {
    float[] radianceArray = wien(lam, t, pOut, pIn).toFloatArray();
    lambdas = ArrayUtils.addAll(lambdas, lamArray);
    radiances = ArrayUtils.addAll(radiances, radianceArray);
    String tString = String.format("T=%dK", (int) t);
    String[] tArray = new String[lamArray.length];
    Arrays.fill(tArray, tString);
    T = ArrayUtils.addAll(T, tArray);
}

Table data = Table.create("data")
    .addColumns(
        DoubleColumn.create("lambda", Functions.floatToDoubleArray(lambdas)),
        DoubleColumn.create("radiance", Functions.floatToDoubleArray(radiances)),
        StringColumn.create("T", T)
);
LinePlot.create("radiance vs. lambda", data, "lambda", "radiance", "T");


Next we assume that we are a fearless physicist measuring some data. Of course, we need to pretend that we don't really know the temperature. But we measure the radiation at a few wavelengths.  

In [ ]:
// Real temperature is approximately 0C
float realtemp = 273;
// We observe at 3000nm up to 20,000nm wavelength
var wavelengths = manager.arange(3,20,2f);
float[] wavelengthArray = wavelengths.toFloatArray();
// Our infrared filters are pretty lousy ...
var delta = manager.randomNormal(new Shape(wavelengths.size()));

var radiance = wien(wavelengths.add(delta), realtemp);
var radianceTrue = wien(wavelengths, realtemp);

float[] wavelengthData = new float[0];
float[] radianceData = new float[0];
String[] typeData = new String[0];

wavelengthData = ArrayUtils.addAll(wavelengthArray, wavelengthArray);
radianceData = ArrayUtils.addAll(radiance.toFloatArray(), radianceTrue.toFloatArray());
String[] measuredType = new String[wavelengthArray.length];
Arrays.fill(measuredType, "measured");
String[] trueType = new String[wavelengthArray.length];
Arrays.fill(trueType, "true");
typeData = ArrayUtils.addAll(measuredType, trueType);

Table data = Table.create("data")
    .addColumns(
        DoubleColumn.create("wavelength", Functions.floatToDoubleArray(wavelengthData)),
        DoubleColumn.create("radiance", Functions.floatToDoubleArray(radianceData)),
        StringColumn.create("type", typeData)
);
LinePlot.create("radiance vs. wavelength", data, "wavelength", "radiance", "type");

Use DJL to estimate the real temperature based on the variables `wavelengths` and `radiance`. 

* You can use Wien's law implementation `wien(lam,t)` as your forward model. 
* Use the loss function $l(y,y') = (\log y - \log y')^2$ to measure accuracy.